In [1]:
import pyforest
import sys
sys.path.append('/Users/jayceepang/msse/ATOM_CODE/UTILS/')
from RF_functions import *
from GP_functions import *
from GenUtils import *
from dataset import *
import os
import pickle
from imblearn.over_sampling import SMOTEN, ADASYN, SMOTE
from sklearn.metrics import accuracy_score, balanced_accuracy_score,precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix,matthews_corrcoef

from rdkit import Chem
from rdkit.Chem import AllChem
import math
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import os

import shutil
import sklearn
from sklearn.model_selection import KFold
import gpytorch
from gpytorch.models import ExactGP
from gpytorch.likelihoods import DirichletClassificationLikelihood
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel, MaternKernel

from sklearn.metrics import confusion_matrix
import itertools
from sklearn.metrics import precision_score, recall_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score, confusion_matrix, f1_score, roc_curve,precision_recall_curve, auc
# from scipy.stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix

import sys

# from RF_GSCV import * # RF_GSCV contains the calculate metrics function to get the TP, TN, FP, FN scores 
# from RF_atomver import prediction_type 


In [9]:
og_neks = ['NEK2_binding','NEK2_inhibition','NEK3_binding','NEK5_binding','NEK9_binding','NEK9_inhibition']
samplings =['none_scaled','UNDER','SMOTE','ADASYN'] 
new_neks=['NEK1', 'NEK11', 'NEK2', 'NEK3', 'NEK4', 'NEK5',
       'NEK6', 'NEK9']
og_datapath = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/'
results_dir='/Users/jayceepang/msse/ATOM_CODE/examples/holdout/holdout_results/'
gp_results_dir='/Users/jayceepang/msse/ATOM_CODE/examples/holdout/holdout_results/GP/'
datapath = '/Users/jayceepang/msse/ATOM_CODE/datasets/holdout_data/featurized/'
gp_path ='/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/GP_results/'
test_results=[]
final_cols=['model','NEK','strategy','feat_type','RF_type', 'cm','recall', 'specificity', 'accuracy', 'precision', 
                'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy']
feat_cols_dropped =['NEK','compound_id','base_rdkit_smiles', 'subset', 'active'] 

In [3]:
def load_and_predict(model_path, likelihood_path, testX, testy, kernel='matern'):
    """
    Get pickled model+likelihood and run predictions new datasets

    @args:
        model_path (str): path topickled model file
        likelihood_path (str): Path to the pickled likelihood file.
        testX (torch.Tensor), testy (torch.Tensor)
        kernel (str): Kernel type RBF or matern
    @returns
        test_perf_df (pd.DataFrame): Prediction results and metrics on test data.
    """
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    with open(likelihood_path, 'rb') as f:
        likelihood = pickle.load(f)

    trainer = Trainer(model, likelihood, iterations=300)
    test_dist, test_observed_pred, test_pred_means, test_pred = trainer.predict(testX)
    test_results = trainer.gp_results(testX, testy)

    test_pred_samples = test_dist.sample(torch.Size((100,))).exp()

    test_probabilities = (test_pred_samples / test_pred_samples.sum(-2, keepdim=True)).mean(0)
    test_prob_stds = (test_pred_samples / test_pred_samples.sum(-2, keepdim=True)).std(0)

    test_perf_df = pd.DataFrame()
    test_perf_df['mean_pred_class0'] = test_observed_pred.mean.numpy()[0,]
    test_perf_df['mean_pred_class1'] = test_observed_pred.mean.numpy()[1,]
    test_perf_df['y'] = testy
    test_perf_df['y_pred'] = test_pred_means.max(0)[1]
    test_perf_df['var_pred_class0'] = test_observed_pred.variance.numpy()[0,]
    test_perf_df['var_pred_class1'] = test_observed_pred.variance.numpy()[1,]
    test_perf_df['pred_prob_class0'] = test_probabilities.numpy()[0,]
    test_perf_df['pred_prob_class1'] = test_probabilities.numpy()[1,]
    test_perf_df['pred_prob_std_class0'] = test_prob_stds.numpy()[0,]
    test_perf_df['pred_prob_std_class1'] = test_prob_stds.numpy()[1,]
    test_perf_df['subset'] = 'nanobret_cellular_holdout'

    for k, val in test_results.items():
        test_perf_df[k] = val

    return test_perf_df


In [11]:
test_results=[]
final_cols=['model','NEK','strategy','feat_type','RF_type', 'cm','recall', 'specificity', 'accuracy', 'precision', 
                'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy']
for nek in new_neks: 
    for i in og_neks: 
        for samp in samplings: 
            trainX=pd.read_csv(f'{og_datapath}{i}_MFP_{samp}_trainX.csv')
            train_y =pd.read_csv(f'{og_datapath}{i}_MFP_{samp}_train_y.csv').to_numpy().reshape(-1)
            testX=pd.read_csv(f'{datapath}{nek}_MFP_none_unscaled.csv').drop(columns=feat_cols_dropped)
            test_y=pd.read_csv(f'{datapath}{nek}_MFP_none_unscaled.csv')['active'].to_numpy().reshape(-1)
            model_name = f'{nek}_MFP_none_scaled'
            # _,_, testX,testy=make_torch_tens_float(filepath=None, filename=None, rootname=None, df=None)
            textX, testy = make_torch_tens_float_simple(file_path_X=None,file_path_y=None, df_path=f'{datapath}{nek}_MFP_none_unscaled.csv')
            gp_model = f'{gp_path}{i}_MFP_none_scaled_matern.pkl'
     
            gp_likelihood = f'{gp_path}{i}_MFP_none_scaled_matern_likelihood.pkl'
            test_df = load_and_predict(gp_model,gp_likelihood, testX,testy, kernel='matern')
            test_df['model'] = f'{model_name}_{i}_GP_matern'
            test_df['NEK'] =nek
            test_df['feat_type'] = 'MFP'
            test_df['strategy'] = samp
            test_df['kernel_type'] = 'GP_matern'
            test_df['prod_model'] = f'{i}_GP_matern'
            final_cols.append('prod_model')
            test_results.append(test_df.iloc[[0]][final_cols].values.flatten())
            
            test_df.to_csv(f'{gp_results_dir}{model_name}_GP_nanobret_cellular_holdout.csv', index=False)


            
    all_test =  pd.DataFrame(test_results,columns=final_cols)
    all_test['modeling_type'] = 'GP' 
    all_test['set'] = 'nanobret_cellular_holdout' 

    # all_test.to_csv(f'{results_dir}RF_holdout_test_results_all_NEK.csv', index=False)                 
         

AttributeError: 'DataFrame' object has no attribute 'ndimension'